In [1]:
# ----------------------------------------------------------------------------
#
# TITLE - linear_model_prior.ipynb
# AUTHOR - James Lane
# PROJECT - AST 1501
#
# ----------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Determine the best-fitting linear model with the addition of a prior
'''

__author__ = "James Lane"

In [11]:
# General
import numpy as np
import pdb, sys, importlib

## Plotting
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation
from matplotlib import colors
from matplotlib import cm

## Astropy
from astropy import units as apu
from astropy.io import fits
from astropy.table import Table

## Scipy
from scipy.stats import binned_statistic_2d, binned_statistic
from scipy.optimize import curve_fit
from scipy import linalg
import scipy as sc

## Galpy
from galpy import potential

## Add project-specific package. Assume relative location
sys.path.append('../../src/')
import ast1501.util
import ast1501.fourier
import ast1501.linear_model
importlib.reload(ast1501.linear_model)

<module 'ast1501.linear_model' from '../../src/ast1501/linear_model.py'>

In [3]:
### Matplotlib for notebooks
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Meta

## Information:
- paper: https://arxiv.org/abs/1008.4686

## Future Work:
- Make sure that all work in the notebook is done using a single set of bootstraps

# About

We assume that the velocity fluctuations induced by the triaxial halo obeys the following restrictions:
- Only produces power in an m=2 mode
- Phase is constant with radius
- Smooth radial trend in power that tends to increase

Fit a simple linear model:

$
v_{T}(R,\phi) = v_{T,0}(R) + A_{T}(R) \cos \big[ 2( \phi - \phi_{b} ) \big] + N_{T}(R)
$

$
v_{R}(R,\phi) =  A_{R}(R) \sin \big[ 2( \phi - \phi_{b} ) \big] + N_{R}(R)
$

See notebook marginalize_linear_fit.ipynb or section below for more information about the linear algebra solution to this problem

Also now add a Gaussian prior on the system. 

$
\exp \big[ ( {\bf X} - {\bf X}_{0} )^{T} {\bf \Sigma}^{-1} ( {\bf X} - {\bf X}_{0} ) \big]
$

Here, ${\bf X}_{0}$ is the column vector prior for each parameter in the linear fit. ${\bf \Sigma}$ is the standard deviation of the Gaussian prior. For a uniform prior, the width of the prior approaches infinity. Therefore this can be accomplished by setting the element in the inverse variance matrix equal to 0.

- Determine a fixed $\phi_{b}$, subject to a prior.
- Calculate $m=2$ amplitudes as a function radius, subject to a prior.
- Compare $m=2$ amplitudes with models

# Prepare

## Load Data

In [4]:
### Load catalogs
gaiadr2_apogee_catalog = '../../data/generated/gaiadr2-apogee_dataset.FIT'
f = fits.open(gaiadr2_apogee_catalog)
data = f[1].data

In [5]:
### Cut on galactocentric absolute Z < 0.3 kpc
where_low_z = np.where( np.abs(data['Z']) < 0.3 )[0]
data_low_z = data[where_low_z] 
z_select_text = r'$|$Z$_{GC}| < 0.3$ kpc'

In [6]:
### Read catalog values

# ID, RA, Dec, logg, abundances, errors
apid = data_low_z['APOGEE_ID']
locid = data_low_z['LOCATION_ID']
vhelio = data_low_z['VHELIO']
pmll = data_low_z['PM_LL']
pmbb = data_low_z['PM_BB']
gc_x = data_low_z['X']
gc_y = data_low_z['Y']
gc_z = data_low_z['Z']
gc_vx = data_low_z['VX']
gc_vy = data_low_z['VY']
gc_vz = data_low_z['VZ']

In [7]:
### Convert to galactocentric radius and radial velocity
gc_R = np.sqrt(np.square(gc_x)+np.square(gc_y))
gc_phi = np.arctan2(gc_y,gc_x)
gc_vR = np.cos(gc_phi)*gc_vx + np.sin(gc_phi)*gc_vy
gc_vT = np.sin(gc_phi)*gc_vx - np.cos(gc_phi)*gc_vy
gc_phi = np.arctan2(gc_y,-gc_x)

## Set Parameters

In [8]:
# Radial bin range and size
R_lims = [5,15]
R_bin_size = 1.0
R_bin_cents = np.arange( R_lims[0], R_lims[1], R_bin_size ) + R_bin_size/2
n_R_bins = len(R_bin_cents)

# Phi bin range and size
phi_lims = [-np.pi/2, np.pi/2]
phi_bin_size = np.pi/30
phi_bin_cents = np.arange( phi_lims[0], phi_lims[1], phi_bin_size ) + phi_bin_size/2
n_phi_bins = len(phi_bin_cents)

# Phib bin range and size
phib_lims = [0, np.pi/2]
phib_bin_size = np.pi/30
phib_bin_cents = np.arange( phib_lims[0], phib_lims[1], phib_bin_size ) + phi_bin_size/2
n_phib_bins = len(phib_bin_cents)

## Generate Bootstrap Sample

In [12]:
make_new_bs_sample = True
if not make_new_bs_sample:
    sys.exit('Not making a new bootstrap sample!')
##fi

n_bs = 100
bs_samples_vR, bs_samples_vT = ast1501.linear_model.make_bootstrap_samples(gc_R, gc_phi, 
    gc_vR, gc_vT, R_bin_cents, R_bin_size, phi_bin_cents, phi_bin_size, n_bs)

# Evaluate - No Iteration

## Tangential Velocity

In [ ]:
fig = plt.figure( figsize=(5,20) )
axs = fig.subplots( nrows=n_R_bins+1, ncols=1 )

store_likelihood_vT = np.ones( ( n_R_bins, n_phib_bins ) )

for i in range(n_R_bins):
    
    likelihood = calculate_phib_likelihood(R_bin_cents[i], R_bin_size, phi_bin_cents, phi_bin_size, 
                                           phib_bin_cents, bs_samples_vT[i], 'vT',
                                           force_yint_zero=False, trig_function=np.cos)
    
    store_likelihood_vT[i,:] = np.log(likelihood)

    # Plot
    axs[i].plot( phib_bin_cents, np.log(likelihood), marker='o', color='Black' )
    axs[i].annotate('R='+str(R_bin_cents[i])+' kpc', fontsize=14, xy=(0.7,0.85), xycoords='axes fraction')
    axs[i].set_ylabel(r'$\ln \mathcal{L}$', fontsize=16)
    axs[i].tick_params(labelbottom='off')
    ##fi

# Marginalize over all radii
prod_likelihood_vT = np.sum(store_likelihood_vT, axis=0)

# Plot
axs[-1].plot( phib_bin_cents, prod_likelihood_vT, marker='s', color='Black' )
axs[-1].annotate('Product', fontsize=14, xy=(0.7,0.85), xycoords='axes fraction')
axs[-1].set_ylabel(r'$\ln \mathcal{L}$', fontsize=16) 

#
axs[-1].set_xlabel(r'$\Phi_{gc}$ (radians)', fontsize=16)
fig.subplots_adjust(hspace=0)
plt.show()
pass;

## Radial velocity

In [ ]:
fig = plt.figure( figsize=(5,20) )
axs = fig.subplots( nrows=n_R_bins+1, ncols=1 )

store_likelihood_vR = np.ones( ( n_R_bins, n_phib_bins ) )

for i in range(n_R_bins):
    
    likelihood = calculate_phib_likelihood(R_bin_cents[i], R_bin_size, phi_bin_cents, phi_bin_size, 
                                           phib_bin_cents, bs_samples_vR[i], 'vR',
                                           force_yint_zero=False, trig_function=np.sin)
    
    store_likelihood_vR[i,:] = np.log(likelihood)
        
    # Plot
    axs[i].plot( phib_bin_cents, np.log(likelihood), marker='o', color='Black' )
    axs[i].annotate('R='+str(R_bin_cents[i])+' kpc', fontsize=14, xy=(0.7,0.85), xycoords='axes fraction')
    axs[i].set_ylabel(r'$\ln \mathcal{L}$', fontsize=16)
    axs[i].tick_params(labelbottom='off')
    ##fi

prod_likelihood_vR = np.sum(store_likelihood_vR, axis=0)

axs[-1].plot( phib_bin_cents, prod_likelihood_vR, marker='s', color='Black' )
axs[-1].annotate('Product', fontsize=14, xy=(0.7,0.85), xycoords='axes fraction')
axs[-1].set_ylabel(r'$\ln \mathcal{L}$', fontsize=16)     

axs[-1].set_xlabel(r'$\Phi_{gc}$ (radians)', fontsize=16)
fig.subplots_adjust(hspace=0)
plt.show()
pass;

## Combine Both Velocities

In [ ]:
fig = plt.figure( figsize=(5,20) )
axs = fig.subplots( nrows=n_R_bins+1, ncols=1 )

store_likelihood_vR = np.ones( ( n_R_bins, n_phib_bins ) )
store_likelihood_vT = np.ones( ( n_R_bins, n_phib_bins ) )

for i in range(n_R_bins):
    
    likelihood_vT = calculate_phib_likelihood(R_bin_cents[i], R_bin_size, phi_bin_cents, phi_bin_size, 
                                              phib_bin_cents, bs_samples_vT[i], 'vT',
                                              force_yint_zero=False, trig_function=np.cos)
    store_likelihood_vT[i,:] = np.log(likelihood_vT)
    
    likelihood_vR = calculate_phib_likelihood(R_bin_cents[i], R_bin_size, phi_bin_cents, phi_bin_size, 
                                              phib_bin_cents, bs_samples_vR[i], 'vR',
                                              force_yint_zero=False, trig_function=np.sin)
    store_likelihood_vR[i,:] = np.log(likelihood_vR)
        
    # Plot
    axs[i].plot( phib_bin_cents, np.log(likelihood_vT/np.amax(likelihood_vT)), marker='o', color='DodgerBlue', alpha=0.75 )
    axs[i].plot( phib_bin_cents, np.log(likelihood_vR/np.amax(likelihood_vR)), marker='o', color='Red', alpha=0.75 )
    axs[i].annotate('R='+str(R_bin_cents[i])+' kpc', fontsize=14, xy=(0.7,0.85), xycoords='axes fraction')
    axs[i].set_ylabel(r'$\ln \mathcal{L}$', fontsize=16)
    axs[i].axvline( np.pi*25/180, color='Black', linestyle='dashed', alpha=0.5 )
    axs[i].tick_params(labelbottom='off')
    ##fi

axs[0].plot([], [], color='DodgerBlue', label=r'$v_{T}$')
axs[0].plot([], [], color='Red', label=r'$v_{R}$')
axs[0].legend(loc=(0.3,0.6))
    
prod_likelihood_vT = np.sum(store_likelihood_vT, axis=0)
prod_likelihood_vR = np.sum(store_likelihood_vR, axis=0)
prod_likelihood_both = prod_likelihood_vR + prod_likelihood_vT

axs[-1].plot( phib_bin_cents, prod_likelihood_vT-np.amax(prod_likelihood_vT), marker='s', color='DodgerBlue', alpha=0.75 )
axs[-1].plot( phib_bin_cents, prod_likelihood_vR-np.amax(prod_likelihood_vR), marker='s', color='Red', alpha=0.75 )
axs[-1].plot( phib_bin_cents, prod_likelihood_both-np.amax(prod_likelihood_both), marker='s', color='Purple', alpha=0.75 )
axs[-1].annotate('Product', fontsize=14, xy=(0.7,0.85), xycoords='axes fraction')
axs[-1].set_ylabel(r'$\ln \mathcal{L}$', fontsize=16)
axs[-1].axvline( np.pi*25/180, color='Black', linestyle='dashed', alpha=0.5 )

axs[-1].set_xlabel(r'$\Phi_{gc}$ (radians)', fontsize=16)
fig.subplots_adjust(hspace=0)
plt.show()
pass;

## Determine Best Fit
Use the combined likelihood to determine the best-fitting radial profile

Since we now know $\phi_{b}$ we can simply use the linear algebra solution to determine the amplitude coeffients in each radial bin

### Tangential Velocity Solution

In [ ]:
# Figure out the maximum likelihood
where_likelihood_max = np.argmax(prod_likelihood_both)
phib_max = phib_bin_cents[ where_likelihood_max ]

fig = plt.figure( figsize=(8,4) )
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

for i in range(n_R_bins):
    X, SIG_X = ast1501.linear_model.calculate_best_fit_m_b(R_bin_cents[i], phib_max, bs_samples_vT[i], 'vT', force_yint_zero=False, 
                                      trig_function=np.cos)
    
    ax1.scatter(R_bin_cents[i], X[0], 
                 edgecolor='DodgerBlue', facecolor='None')
    ax2.scatter(R_bin_cents[i], np.abs(X[1]),
                 edgecolor='DodgerBlue', facecolor='None')
    ax1.errorbar(R_bin_cents[i], X[0], yerr=np.sqrt(SIG_X[0,0]), 
                 markeredgecolor='DodgerBlue', markerfacecolor='None', ecolor='Black')
    ax2.errorbar(R_bin_cents[i], np.abs(X[1]), yerr=np.sqrt(SIG_X[1,1]),
                 markeredgecolor='DodgerBlue', markerfacecolor='None', ecolor='Black')
###i

ax1.set_xlabel('R [kpc]')
ax1.set_ylabel(r'b $(v_{c})$ [km/s]')

ax2.set_xlabel('R [kpc]')
ax2.set_ylabel(r'|m| (A) [km/s]')

plt.show()
pass;

In [ ]:
vT_amps = np.zeros(n_R_bins)
vT_bs = np.zeros(n_R_bins)

for i in range(n_R_bins):
    X, SIG_X = ast1501.linear_model.calculate_best_fit_m_b(R_bin_cents[i], phib_max, bs_samples_vT[i], 'vT', force_yint_zero=False, 
                                      trig_function=np.cos)
    vT_bs[i] = X[0]
    vT_amps[i] = X[1]
###i
                              
fig, axs = ast1501.linear_model.radial_velocity_known_m_b_phi( R_bin_cents, R_bin_size, phi_lims, phi_bin_size, gc_R, gc_phi, gc_vT,
                            vT_amps, vT_bs, phib_max, np.cos,
                            phi_bin_size_in_arc=False)

for i in range( len(R_bin_cents) ):
    axs[i,0].set_ylabel(r'$v_{T}- \langle v_{T} \rangle$ [km/s]')
##ax

fig.subplots_adjust( wspace=0.2, hspace=0.3 )

plt.show()

### Radial Velocity Solution

In [ ]:
# Figure out the maximum likelihood
where_likelihood_max = np.argmax(prod_likelihood_both)
phib_max = phib_bin_cents[ where_likelihood_max ]

fig = plt.figure( figsize=(8,4) )
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

for i in range(n_R_bins):
    X, SIG_X = calculate_best_fit_m_b(R_bin_cents[i], phib_max, bs_samples_vR[i], 'vR', force_yint_zero=False, 
                                      trig_function=np.sin)
    
    ax1.scatter(R_bin_cents[i], X[0], 
                 edgecolor='DodgerBlue', facecolor='None')
    ax2.scatter(R_bin_cents[i], np.abs(X[1]),
                 edgecolor='DodgerBlue', facecolor='None')
    ax1.errorbar(R_bin_cents[i], X[0], yerr=np.sqrt(SIG_X[0,0]), 
                 markeredgecolor='DodgerBlue', markerfacecolor='None', ecolor='Black')
    ax2.errorbar(R_bin_cents[i], np.abs(X[1]), yerr=np.sqrt(SIG_X[1,1]),
                 markeredgecolor='DodgerBlue', markerfacecolor='None', ecolor='Black')
###i

ax1.set_xlabel('R [kpc]')
ax1.set_ylabel(r'b [km/s]')

ax2.set_xlabel('R [kpc]')
ax2.set_ylabel(r'|m| (A) [km/s]')

plt.show()
pass;

In [ ]:
vR_amps = np.zeros(n_R_bins)
vR_bs = np.zeros(n_R_bins)

for i in range(n_R_bins):
    X, SIG_X = calculate_best_fit_m_b(R_bin_cents[i], phib_max, bs_samples_vR[i], 'vR', force_yint_zero=False, 
                                      trig_function=np.sin)
    vR_bs[i] = X[0]
    vR_amps[i] = X[1]
###i
                              
fig, axs = radial_velocity_known_m_b_phi( R_bin_cents, R_bin_size, phi_lims, phi_bin_size, gc_R, gc_phi, gc_vR,
                            vR_amps, vR_bs, phib_max, np.sin,
                            phi_bin_size_in_arc=False)

for i in range( len(R_bin_cents) ):
    axs[i,0].set_ylabel(r'$v_{R}- \langle v_{R} \rangle$ [km/s]')
##ax

fig.subplots_adjust( wspace=0.2, hspace=0.3 )

plt.show()

# Evaluate - Iteration
Add noise into the model by comparing the best-fitting sinusoids with the data, then using that information to generate an extra noise term in each radial bin

In [ ]:
n_R_bins = len(R_bin_cents)
n_phi_bins = len(phi_bin_cents)
n_phib_bins = len(phib_bin_cents)
extra_variance = np.zeros((n_R_bins,2))
n_iterate = 5

alpha_low = 0.25
alpha_increment = (1-alpha_low)/(n_iterate-1)

fig1 = plt.figure( figsize=(5,20) )
axs1 = fig1.subplots( nrows=n_R_bins+1, ncols=1 )

fig2 = plt.figure( figsize=(12,6) ) 
axs2 = fig2.subplots( nrows=2, ncols=2 )

for i in range( n_iterate ):
    
    store_likelihood_vT, store_likelihood_vR, prod_likelihood_vT, prod_likelihood_vR,\
    prod_likelihood_both, phib_max_likelihood, bs, ms, bs_err, ms_err, variance_model_data\
    = iterate_noise_model(R_bin_cents, R_bin_size, phi_bin_cents, phi_bin_size, 
                          phib_bin_cents, bs_samples_vT, bs_samples_vR, extra_variance)
    
    extra_variance = variance_model_data
    
    for j in range( n_R_bins ):
        axs1[j].plot( phib_bin_cents, store_likelihood_vT[j,:], marker='o', markersize=5, color='DodgerBlue', 
                      alpha=alpha_low + i*alpha_increment )
        axs1[j].plot( phib_bin_cents, store_likelihood_vR[j,:], marker='o', markersize=5, color='Red', 
                      alpha=alpha_low + i*alpha_increment )
        axs1[j].annotate('R='+str(R_bin_cents[j])+' kpc', fontsize=14, xy=(0.7,0.85), xycoords='axes fraction')
        axs1[j].set_ylabel(r'$\ln \mathcal{L}$', fontsize=16)
        axs1[j].axvline( np.pi*25/180, color='Black', linestyle='dashed', alpha=0.5 )
        axs1[j].tick_params(labelbottom='off')
    ###i
    
    axs2[0,0].scatter(R_bin_cents, bs[:,0], edgecolor='DodgerBlue', facecolor='DodgerBlue', 
                    alpha=alpha_low + i*alpha_increment )
    axs2[0,1].scatter(R_bin_cents, np.abs(ms)[:,0], edgecolor='DodgerBlue', facecolor='DodgerBlue', 
                    alpha=alpha_low + i*alpha_increment )
    axs2[1,0].scatter(R_bin_cents, bs[:,1], edgecolor='Red', facecolor='Red', 
                    alpha=alpha_low + i*alpha_increment )
    axs2[1,1].scatter(R_bin_cents, np.abs(ms)[:,1], edgecolor='Red', facecolor='Red', 
                    alpha=alpha_low + i*alpha_increment )
    
    if i == n_iterate-1:
        axs2[0,0].errorbar(R_bin_cents, bs[:,0], fmt='o', markeredgecolor='Black', markerfacecolor='DodgerBlue', 
                           alpha=alpha_low + i*alpha_increment, yerr=bs_err[:,0], ecolor='Black')
        axs2[0,1].errorbar(R_bin_cents, np.abs(ms)[:,0], fmt='o', markeredgecolor='Black', markerfacecolor='DodgerBlue', 
                           alpha=alpha_low + i*alpha_increment, yerr=ms_err[:,0], ecolor='Black')
        axs2[1,0].errorbar(R_bin_cents, bs[:,1], fmt='o', markeredgecolor='Black', markerfacecolor='Red', 
                           alpha=alpha_low + i*alpha_increment, yerr=bs_err[:,1], ecolor='Black')
        axs2[1,1].errorbar(R_bin_cents, np.abs(ms)[:,1], fmt='o', markeredgecolor='Black', markerfacecolor='Red', 
                           alpha=alpha_low + i*alpha_increment, yerr=ms_err[:,1], ecolor='Black')
    ###i
    
    if i == 0:
        axs1[0].plot([], [], color='DodgerBlue', label=r'$v_{T}$')
        axs1[0].plot([], [], color='Red', label=r'$v_{R}$')
        axs1[0].legend(loc=(0.3,0.6))
        axs1[-1].annotate('Product', fontsize=14, xy=(0.7,0.85), xycoords='axes fraction')
        axs1[-1].set_ylabel(r'$\ln \mathcal{L}$', fontsize=16)
        axs1[-1].axvline( np.pi*25/180, color='Black', linestyle='dashed', alpha=0.5 )
        axs1[-1].set_xlabel(r'$\Phi_{gc}$ (radians)', fontsize=16)
        
        axs2[0,0].set_xlabel('R [kpc]')
        axs2[0,0].set_ylabel(r'b $(v_{c})$ [km/s]')
        axs2[0,1].set_xlabel('R [kpc]')
        axs2[0,1].set_ylabel(r'|m| (A) [km/s]')
        axs2[1,1].set_xlabel('R [kpc]')
        axs2[1,1].set_ylabel(r'|m| (A) [km/s]')
        axs2[1,0].set_xlabel('R [kpc]')
        axs2[1,0].set_ylabel(r'b $(v_{c})$ [km/s]')
    ###j

    axs1[-1].plot( phib_bin_cents, prod_likelihood_vT, marker='s', markersize=5, color='DodgerBlue', 
                   alpha=alpha_low + i*alpha_increment)
    axs1[-1].plot( phib_bin_cents, prod_likelihood_vR, marker='s', markersize=5, color='Red', 
                   alpha=alpha_low + i*alpha_increment)
    axs1[-1].plot( phib_bin_cents, prod_likelihood_both, marker='s', markersize=5, color='Purple', 
                   alpha=alpha_low + i*alpha_increment)
    
    # Overplot the circular velocity curve
    vcirc_Rs = np.linspace(5,15,num=100)
    vcirc = potential.vcirc(potential.MWPotential2014,R=vcirc_Rs/8)*220 # km/s
    axs2[0,0].plot(vcirc_Rs, vcirc, linewidth=1.0, color='Black', alpha=0.5, linestyle='dashed')
    
fig1.subplots_adjust(hspace=0)
plt.show()
pass;

# Vector Math

The solution to the equation with the $\chi^{2}$ term is:

$
\chi^2 = ({\bf X} - {\bf W})^T {\bf V}^{-1}\,({\bf X} - {\bf W}) + {\bf U}
$

with

$
W = [A^{T}C^{-1}A]^{-1}A^{T}C^{-1}Y
$

$
W^{T} = Y^{T}C^{-1}A[A^{T}C^{-1}A]^{-1}
$

$
V^{-1} = A^{T}C^{-1}A
$

and

$
U = Y^{T}C^{-1}[Y-AW]
$

The solution to the integral is:

$
p( \phi_{b} | Data ) \propto \sqrt{ \frac{ \det V }{ \det \Sigma } }\exp[-U/2]
$

First, create $\mathbf{A}$, a $N \times 2$ matric of $1$ and $\cos[2(\phi-\phi_{b})]$. Also create $\mathbf{Y}$, a column vector of velocity data points. Finally, make the covariance matrix after bootstrapping in bins.

In [ ]:
# Find all the points within this radial bin
def determine_best_fitting_m_b(gc_R, gc_phi, gc_v, R_bin_cent, R_bin_size, phib, n_bs=10, force_yint_zero=False,
                              trig_function=np.cos, zero_vels=False):
    '''determine_best_fitting_m_b:
    
    Args:
    
    Returns:
    
    '''
    stars_in_R_bin = np.where( ( gc_R < R_bin_cent + R_bin_size/2 ) & 
                               ( gc_R > R_bin_cent - R_bin_size/2 ) )[0]
    n_stars_in_R_bin = len(stars_in_R_bin)
    gc_R_in_R_bin = gc_R[stars_in_R_bin]
    gc_phi_in_R_bin = gc_phi[stars_in_R_bin]
    gc_v_in_R_bin = gc_v[stars_in_R_bin]    
    
    phi_bin_v = np.array([])
    phi_bin_v_err = np.array([])
    phi_bin_phi = np.array([])
    phi_bin_phi_err = np.array([])

    # Loop over phi bins
    for j in range(n_phi_bins):

        # Find all the points within this phi bin
        stars_in_phi_bin = np.where( ( gc_phi_in_R_bin < phi_bin_cents[j] + phi_bin_size/2 ) &
                                     ( gc_phi_in_R_bin > phi_bin_cents[j] - phi_bin_size/2 ) )[0]
        n_stars_in_phi_bin = len(stars_in_phi_bin)
        gc_R_in_phi_bin = gc_R_in_R_bin[stars_in_phi_bin]
        gc_phi_in_phi_bin = gc_phi_in_R_bin[stars_in_phi_bin]
        gc_v_in_phi_bin = gc_v_in_R_bin[stars_in_phi_bin]

        # If we have more than a certain number of stars then BS
        bs_v_avg_samps = np.array([])
        bs_phi_avg_samps = np.array([])
        if n_stars_in_phi_bin > 10:

            # Loop over BS samples
            for k in range(n_bs):
                sample = np.random.randint(0,n_stars_in_phi_bin,n_stars_in_phi_bin)
                bs_v_avg_samps = np.append( bs_v_avg_samps, np.average(gc_v_in_phi_bin[sample]) )
                bs_phi_avg_samps = np.append( bs_phi_avg_samps, np.average(gc_phi_in_phi_bin[sample]) )
            ###k
    
            # Append the mean to the list of measurements
            phi_bin_v = np.append( phi_bin_v, np.mean( bs_v_avg_samps ) )
            phi_bin_v_err = np.append( phi_bin_v, np.std( bs_v_avg_samps ) )
            phi_bin_phi = np.append( phi_bin_phi, np.mean( bs_phi_avg_samps ) )
            phi_bin_phi_err = np.append( phi_bin_phi_err, np.std( bs_phi_avg_samps ) )
        ##fi
    ###j
    
    if zero_vels:
        phi_bin_v -= np.nanmean(phi_bin_v)

    # Now make the vectors
    n_good_phi_bins = len(phi_bin_v)
    Y = np.zeros((n_good_phi_bins,1))
    C = np.zeros((n_good_phi_bins,n_good_phi_bins))

    Y[:,0] = phi_bin_v
    for j in range(n_good_phi_bins):
        C[j,j] = phi_bin_v_err[j]**2
    ###j
    C_inv = np.linalg.inv(C)
    
    # Make sure to check if the y intercept is forced to be 0
    if force_yint_zero:
        A = np.ones((n_good_phi_bins,1))
        A[:,0] = trig_function( 2*( phi_bin_phi - phib ) )
    else:
        A = np.ones((n_good_phi_bins,2))
        A[:,1] = trig_function( 2*( phi_bin_phi - phib ) )
    ##ie
    
    X = np.matmul( np.linalg.inv( np.linalg.multi_dot([A.T,np.linalg.inv(C),A]) ), np.linalg.multi_dot([A.T,np.linalg.inv(C),Y]) )
    SIG_X = np.linalg.inv( np.linalg.multi_dot([A.T,np.linalg.inv(C),A]) )
    return X, SIG_X
#def

# Find all the points within this radial bin
def determine_best_fitting_m_b_prior(gc_R, gc_phi, gc_v, R_bin_cent, R_bin_size, phib, n_bs=10, force_yint_zero=False,
                                     trig_function=np.cos, zero_vels=False):
    '''determine_best_fitting_m_b_prior:
    
    Args:
    
    Returns:
    
    '''
    stars_in_R_bin = np.where( ( gc_R < R_bin_cent + R_bin_size/2 ) & 
                               ( gc_R > R_bin_cent - R_bin_size/2 ) )[0]
    n_stars_in_R_bin = len(stars_in_R_bin)
    gc_R_in_R_bin = gc_R[stars_in_R_bin]
    gc_phi_in_R_bin = gc_phi[stars_in_R_bin]
    gc_v_in_R_bin = gc_v[stars_in_R_bin]    
    
    phi_bin_v = np.array([])
    phi_bin_v_err = np.array([])
    phi_bin_phi = np.array([])
    phi_bin_phi_err = np.array([])

    # Loop over phi bins
    for j in range(n_phi_bins):

        # Find all the points within this phi bin
        stars_in_phi_bin = np.where( ( gc_phi_in_R_bin < phi_bin_cents[j] + phi_bin_size/2 ) &
                                     ( gc_phi_in_R_bin > phi_bin_cents[j] - phi_bin_size/2 ) )[0]
        n_stars_in_phi_bin = len(stars_in_phi_bin)
        gc_R_in_phi_bin = gc_R_in_R_bin[stars_in_phi_bin]
        gc_phi_in_phi_bin = gc_phi_in_R_bin[stars_in_phi_bin]
        gc_v_in_phi_bin = gc_v_in_R_bin[stars_in_phi_bin]

        # If we have more than a certain number of stars then BS
        bs_v_avg_samps = np.array([])
        bs_phi_avg_samps = np.array([])
        if n_stars_in_phi_bin > 10:

            # Loop over BS samples
            for k in range(n_bs):
                sample = np.random.randint(0,n_stars_in_phi_bin,n_stars_in_phi_bin)
                bs_v_avg_samps = np.append( bs_v_avg_samps, np.average(gc_v_in_phi_bin[sample]) )
                bs_phi_avg_samps = np.append( bs_phi_avg_samps, np.average(gc_phi_in_phi_bin[sample]) )
            ###k
    
            # Append the mean to the list of measurements
            phi_bin_v = np.append( phi_bin_v, np.mean( bs_v_avg_samps ) )
            phi_bin_v_err = np.append( phi_bin_v, np.std( bs_v_avg_samps ) )
            phi_bin_phi = np.append( phi_bin_phi, np.mean( bs_phi_avg_samps ) )
            phi_bin_phi_err = np.append( phi_bin_phi_err, np.std( bs_phi_avg_samps ) )
        ##fi
    ###j
    
    if zero_vels:
        phi_bin_v -= np.nanmean(phi_bin_v)

    # Now make the vectors
    n_good_phi_bins = len(phi_bin_v)
    Y = np.zeros((n_good_phi_bins,1))
    B = np.zeros((n_good_phi_bins,1))
    C = np.zeros((n_good_phi_bins,n_good_phi_bins))
    CB = np.zeros((n_good_phi_bins,n_good_phi_bins))

    Y[:,0] = phi_bin_v
    B[:,0] = potential.vcirc(potential.MWPotential2014,R=R_bin_cent/8)*220 # km/s
    for j in range(n_good_phi_bins):
        C[j,j] = phi_bin_v_err[j]**2
        CB[j,j] = 5**2 # Fixed error in the potential
    ###j
    C_inv = np.linalg.inv(C)
    C_p_CB_inv = np.linalg.inv(C+CB)
    C_TOT = np.linalg.multi_dot([C,C_p_CB_inv,CB])
    C_TOT_inv = np.linalg.inv(C_TOT)
    
    # Make sure to check if the y intercept is forced to be 0
    if force_yint_zero:
        A = np.ones((n_good_phi_bins,1))
        A[:,0] = trig_function( 2*( phi_bin_phi - phib ) )
    else:
        A = np.ones((n_good_phi_bins,2))
        A[:,1] = trig_function( 2*( phi_bin_phi - phib ) )
    ##ie
    
    X1 = np.linalg.inv( np.linalg.multi_dot( [A.T,
                                              C_p_CB_inv.T,
                                              CB.T,
                                              C_TOT_inv,
                                              C,
                                              C_p_CB_inv,
                                              A
                                             ] ) )
    X2 = np.linalg.multi_dot( [A.T,
                               C_p_CB_inv,
                               CB.T,
                               C_TOT_inv,
                               ( Y-np.linalg.multi_dot([C,C_p_CB_inv,B]) )
                              ] )
    X = np.matmul(X1,X2)
    return X, X1
#def